In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler


In [ ]:
df = pd.read_csv('../data/csv/games_data_2.csv')
team_stats_by_season = pd.read_csv('../data/csv/team_stats.csv')

In [ ]:
team_stats_by_season['% Побед дома'] = team_stats_by_season['Побед дома'].apply(
    lambda x: round(int(x.split('-')[0]) / int(x.split('-')[1]) if int(x.split('-')[1]) != 0 else int(x.split('-')[0]),
                    2)).copy()
team_stats_by_season['% Побед в гостях'] = team_stats_by_season['Побед в гостях'].apply(
    lambda x: round(int(x.split('-')[0]) / int(x.split('-')[1]) if int(x.split('-')[1]) != 0 else int(x.split('-')[0]),
                    2)).copy()
team_stats_by_season.drop(columns=['Побед дома'], inplace=True)
team_stats_by_season.drop(columns=['Побед в гостях'], inplace=True)

In [ ]:
# Выборка нечисловых значений
float_fields = [x for x in df.keys() if x.endswith('3/В') or x.endswith('В прот.')]
percent_fields = [x for x in df.keys() if x.endswith('%') or x.endswith('% прот.')]

In [ ]:
# Представление нечисловых значений в процентах
for fl in float_fields:
    df[fl] = df[fl].apply(lambda x: int(x.split('/')[0]) / int(x.split('/')[1]) if '/' in x else 0)

for per in percent_fields:
    df[per] = df[per].apply(lambda x: int(x) / 100 if x > 0 else 0)

df = df.dropna(axis=0, how='any')

In [ ]:
# Создание столбца Сезон
def get_season(date_str):
    year = int(date_str.split()[-1])
    if year == 2024:
        return 2024
    return f"{year + 1}"


df['Сезон'] = df['Дата'].apply(get_season)

In [ ]:
dataset = pd.merge(df, team_stats_by_season, on=['Имя', 'Сезон'], how='left')
team_stats_by_season_prot = team_stats_by_season.rename(columns=lambda x: x + ' прот.' if x not in ['Сезон', 'Имя'] else x)

# Объединение данных по имени противника
dataset = pd.merge(dataset, team_stats_by_season_prot, left_on=['Имя прот.', 'Сезон'], right_on=['Имя', 'Сезон'], how='left')

In [3]:
# Целевой признак
def add_target(team):
    team['Цель'] = team['Победа'].shift(-1)
    return team

In [ ]:
# Подготовка данных 
dataset = dataset.groupby('Имя_x', group_keys=False).apply(add_target).copy().dropna().reset_index(drop=True)
dataset['Цель'] = dataset['Цель'].astype(int, errors='ignore')

dataset.rename(columns={'Имя_x': 'Имя'}, inplace=True)
dataset.drop(columns=['Имя_y'], inplace=True)

In [ ]:
# Создание модели логистической регрессии
lr = LogisticRegression()
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(lr, n_features_to_select=30, direction='forward', cv=split)

In [ ]:
# Подготовка данных для подгонки
removed_cols = ['Сезон', 'Дата', 'Цель', 'Победа', 'Имя', 'Имя прот.', 'Победа дома']
selected_cols = dataset.columns[~dataset.columns.isin(removed_cols)]

scaler = MinMaxScaler()
dataset[selected_cols] = scaler.fit_transform(dataset[selected_cols])

# Подгонка
sfs.fit(dataset[selected_cols], dataset['Цель'])
predictors = list(selected_cols[sfs.get_support()])

args = ['Блокшоты', 'Забито', 'Перехваты', 'Пропущено', 'Разница']
for i in args:
    predictors.append(i)
predictors.sort()

In [ ]:
# График важности признаков
importance = pd.Series(lr.coef_[0], index=predictors).sort_values(ascending=False)

plt.figure(figsize=(10, 8))
importance.plot(kind='bar')
plt.title('Важность признаков')
plt.xlabel('Признаки')
plt.ylabel('Значение коэффициента')
plt.show()

In [ ]:
# Производительность модели по сезонам
season_accuracy = []

seasons = sorted(dataset['Сезон'].unique())
for season in seasons:
    train = dataset[dataset['Сезон'] < season]
    test = dataset[dataset['Сезон'] == season]

    # Проверяем, что у нас есть достаточно данных для обучения и тестирования
    if len(train) == 0 or len(test) == 0:
        print(f"Недостаточно данных для сезона {season}. Пропускаем этот сезон.")
        continue

    lr.fit(train[predictors], train['Цель'])
    predictions = lr.predict(test[predictors])
    accuracy = accuracy_score(test['Цель'], predictions)
    season_accuracy.append((season, accuracy))


In [ ]:

season_accuracy_df = pd.DataFrame(season_accuracy, columns=['Сезон', 'Точность'])

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x='Сезон', y='Точность', data=season_accuracy_df)
plt.title('Точность модели по сезонам')
plt.xlabel('Сезон')
plt.ylabel('Точность')
plt.show()

In [12]:
# Обучение модели 
def back_test(data, model, predictors, start=2, step=1):
    seasons = sorted(data['Сезон'].unique())
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data['Сезон'] < season]
        model.fit(train[predictors], train['Цель'])
    return model

In [ ]:
model = back_test(dataset, lr, predictors)

In [14]:
# Получение средних показателей команды за последние 10 игр по названиям
def get_team_data(team, team_2):

    team_1_df = dataset[(dataset['Имя'] == team)]
    team_2_df = dataset[(dataset['Имя'] == team_2)]

    team_1_df['Сезон'] = pd.to_numeric(team_1_df['Сезон'])
    team_2_df['Сезон'] = pd.to_numeric(team_2_df['Сезон'])

    team_1_sorted = (team_1_df.sort_values(by='Сезон'))
    team_2_sorted = team_2_df.sort_values(by='Сезон')

    last_10_games_1 = team_1_sorted[predictors].tail(10).filter(regex='^(?!.*прот).*$')
    last_10_games_2 = team_2_sorted[predictors].tail(10).filter(regex='^(?!.*прот).*$').rename(
        columns=lambda x: f"{x} прот.")
    last_10_games_2.drop(columns=['МИН прот.', 'Потери прот.', 'Фолы прот.'], inplace=True)
    combined = pd.concat([last_10_games_1.reset_index(drop=True), last_10_games_2.reset_index(drop=True)], axis=1)
    return combined.mean().to_frame().T.sort_index(axis=1)

In [ ]:
t1 = 'ЦСКА'
t2 = 'Автодор'

pr_data = get_team_data(t1, t2)
pr_data = pr_data.loc[:, ~pr_data.columns.duplicated()]

pred = model.predict_proba(pr_data)
print(f'Вероятность победы в матче: {t1 if pred[0][0] < pred[0][1] else t2} побеждает')
print(pred)

In [ ]:
# Сравнение производительности команд
team_1_data = dataset[dataset['Имя'] == t1].tail(10)
team_2_data = dataset[dataset['Имя'] == t2].tail(10)

team_1_avg = team_1_data[predictors].mean()
team_2_avg = team_2_data[predictors].mean()

comparison_df = pd.DataFrame({'Команда 1': team_1_avg, 'Команда 2': team_2_avg})
comparison_df.plot(kind='bar', figsize=(14, 8))
plt.title(f'Сравнение производительности за последние 10 игр: {t1} vs {t2}')
plt.xlabel('Признаки')
plt.ylabel('Среднее значение')
plt.show()